In [5]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = '63A451E2DA73C4D6A4C64A17B36B9565'
project = Project(api_url, api_key)


In [6]:
#extract study termination data
supplementation_data = project.export_records(forms=['maternal_supplementation'])

#convert data to Dataframe.
supp_data=pd.DataFrame(supplementation_data)

RequestException: {'error': 'You do not have permissions to use the API'}

In [ ]:
# # Specify the fields you want to extract base on logics
# df_suppData=supp_data[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','redcap_repeat_instance','ms_dvisit_q1','ms_sup_administered_q2','ms_completed_by']]
# df_suppData=pd.DataFrame(df_suppData)

# #st_date as a date data type
# df_suppData['ms_dvisit_q1'] = pd.to_datetime(df_suppData['ms_dvisit_q1'], errors='coerce')


# filtered_data = df_suppData[
#     (df_suppData['redcap_event_name'] == 'daily_supplementat_arm_1') &
#     (df_suppData['ms_sup_administered_q2'] == '0')
# ]


In [ ]:
import pandas as pd

# Step 1: Filter and select relevant columns
filtered_data = supp_data[
    (supp_data['redcap_event_name'] == 'daily_supplementat_arm_1') &
    (supp_data['ms_sup_administered_q2'] == '0')
][[
    'con_participantid_q1', 'redcap_event_name', 'redcap_repeat_instrument',
    'redcap_repeat_instance', 'ms_dvisit_q1', 'ms_sup_administered_q2', 'ms_completed_by'
]]

# Step 2: Convert 'ms_dvisit_q1' to date
filtered_data['ms_dvisit_q1'] = pd.to_datetime(filtered_data['ms_dvisit_q1'], errors='coerce').dt.date

# Step 3: Group by participant ID and check for five consecutive days
def has_five_consecutive_days(group):
    # Sort by date
    group = group.sort_values(by='ms_dvisit_q1').reset_index(drop=True)
    
    # Calculate day differences
    group['day_diff'] = (pd.to_datetime(group['ms_dvisit_q1']) - pd.to_datetime(group['ms_dvisit_q1'].shift())).dt.days.fillna(1)
    
    # Identify streaks of consecutive days
    group['streak'] = (group['day_diff'] == 1).cumsum()
    streak_counts = group.groupby('streak').size()
    
    # Check if any streak has 5 or more consecutive days
    if (streak_counts >= 5).any():
        return group
    return None  # Return None explicitly if no valid streaks found

# Apply the logic to each group
results = filtered_data.groupby('con_participantid_q1').apply(has_five_consecutive_days)

# Ensure only valid DataFrames are retained
valid_results = [group for group in results if isinstance(group, pd.DataFrame) and not group.empty]

# Combine valid results into a single DataFrame
if valid_results:
    final_result = pd.concat(valid_results, ignore_index=True)
else:
    final_result = pd.DataFrame(columns=filtered_data.columns)

# Step 4: Return the final filtered data with relevant columns
columns_to_return = [
    'con_participantid_q1', 'redcap_event_name', 'redcap_repeat_instrument',
    'redcap_repeat_instance', 'ms_dvisit_q1', 'ms_sup_administered_q2', 'ms_completed_by'
]

final_result = final_result[columns_to_return]

# Output the result
print(final_result)


Empty DataFrame
Columns: [con_participantid_q1, redcap_event_name, redcap_repeat_instrument, redcap_repeat_instance, ms_dvisit_q1, ms_sup_administered_q2, ms_completed_by]
Index: []


In [ ]:
# #Daily supplementation date diffference
# df_supp = pd.DataFrame({
#     'ID': df_suppData['con_participantid_q1'].shift(-1)+"-"+(df_supp['redcap_repeat_instance'] + 1).astype(str)+"-"+(pd.to_datetime(df_suppData['ms_dvisit_q1'].shift(-1))).astype(str),
#     'con_participantid_q1': df_suppData['con_participantid_q1'].shift(-1),
#     'Instance number': df_suppData['redcap_repeat_instance'].shift(-1),
#     'Supplementation date': pd.to_datetime(df_suppData['ms_dvisit_q1'].shift(-1)),
#     'Instrument': 'Maternal Supplementation',
#     'Completed by' : df_suppData['ms_completed_by'].shift(-1),
#     'Description':  'This date difference between instance ' + df_supp['redcap_repeat_instance'].astype(str) + ' and instance ' + (df_supp['redcap_repeat_instance'] + 1).astype(str) + ' is not equals to 1.'

# })

In [ ]:
# #Save the filtered DataFrame to a CSV file
# df_supp.to_csv('SupplementationQuery_PD.csv', index=False)